In [31]:
import numpy as np
from numba import jit, vectorize, float32, float64, autojit
from scipy.spatial import cKDTree
from matplotlib import pyplot as plt

def GetHsml(coords, des_ngb=32, boxsize=False):
    if not boxsize:
        tree = cKDTree(coords)
    else:
        tree = cKDTree((coords+boxsize/2)%boxsize, boxsize=boxsize)
    neighbor_dists, neighbors = tree.query(coords, des_ngb)
    if len(coords) < des_ngb:
        return np.ones(len(coords))*neighbor_dists.max()
    hsml = GetHsmlWork(neighbors, neighbor_dists)
    return hsml

@jit
def GetHsmlWork(neighbors, neighbor_dists):
    N, des_ngb = neighbor_dists.shape
    hsml = np.zeros(N)
    n_ngb = 0.0
    for i in xrange(N):
        upper = neighbor_dists[i,des_ngb-1]/0.6
        lower = neighbor_dists[i,1]
        error = 1e100
        count = 0
        while error > 1e-3:
            h = (upper + lower)/2
            n_ngb=0.0
            dngb=0.0
            q = 0.0
            for j in xrange(des_ngb):
                q = neighbor_dists[i, j]/h
                if q <= 0.5:
                    n_ngb += (1 - 6*q**2 + 6*q**3)
                elif q <= 1.0:
                    n_ngb += 2*(1-q)**3
            n_ngb *= 32./3
            if n_ngb > des_ngb:
                upper = h
            else:
                lower = h
            error = np.fabs(n_ngb-des_ngb)
        hsml[i] = h
    return hsml

@vectorize([float32(float32), float64(float64)])
def Kernel3D(q):
    if q <= 0.5:
        return 1.8189136353359467 * (1 - 6*q**2 + 6*q**3)
    elif q <= 1.0:
        return 1.8189136353359467 * 2 * (1-q)**3
    else: return 0.0
    
@vectorize([float32(float32), float64(float64)])
def Kernel2D(q):
    if q <= 0.5:
        return 1.8189136353359467 * (1 - 6*q**2 + 6*q**3)
    elif q <= 1.0:
        return 1.8189136353359467 * 2 * (1-q)**3
    else: return 0.0

@jit#("f8[:,:,:](f8[:,:], f8[:], i8, f8[:], f8)")
def GridDensity(data, x, h, gridres, L):
    if len(data.shape) == 1:
        data  = np.array([data,])
    grid = np.zeros((gridres,gridres,len(data)))
    nfields = len(data)
    dx = L/(gridres-1)
    N = len(x)
    for i in xrange(N):
        xs = x[i] + L/2
        hs = h[i]

        gxmin = max(int((xs[0] - hs)/dx + 0.5),0)
        gxmax = min(int((xs[0] + hs)/dx) + 1,gridres)
        gymin = max(int((xs[1] - hs)/dx + 0.5), 0)
        gymax = min(int((xs[1] + hs)/dx) + 1, gridres)
        
        for gx in xrange(gxmin, gxmax):
            for gy in xrange(gymin,gymax):
                kernel = Kernel2D(((xs[0] - gx*dx)**2 + (xs[1] - gy*dx)**2)**0.5 / hs)
                for k in xrange(nfields):
                    grid[gx,gy,k] += data[k,i]*kernel / hs**2
                    
    return grid
    
@jit("f8[:,:](f8[:], f8[:], f8[:], i8, f8)")
def IntegrateSurfaceDensity(mass, x, h, gridres, L):
    grid = np.zeros((gridres,gridres))
    dx = L/(gridres-1)
    N = len(x)
    for i in xrange(N):
        xs = x[i] + L/2
        hs = h[i]
        mh2 = mass[i]/hs**2

        gxmin = max(int((xs[0] - hs)/dx+1),0)
        gxmax = min(int((xs[0] + hs)/dx),gridres-1)
        gymin = max(int((xs[1] - hs)/dx+1), 0)
        gymax = min(int((xs[1] + hs)/dx), gridres-1)
        
        for gx in xrange(gxmin, gxmax+1):
            for gy in xrange(gymin,gymax+1):
                kernel = Kernel2D(((xs[0] - gx*dx)**2 + (xs[1] - gy*dx)**2)**0.5 / hs)
                grid[gx,gy] +=  kernel * mh2
                
    return grid

@jit("f8[:,:](f8[:], f8[:], f8[:], i8, f8)")
def IntegrateSurfaceDensityPeriodic(mass, x, h, gridres, L):
    x = (x-L/2)%L
    grid = np.zeros((gridres,gridres))
    dx = L/(gridres-1)
    N = len(x)
    for i in xrange(N):
        xs = x[i]
        hs = h[i]
        mh2 = mass[i]/hs**2

        gxmin = int((xs[0] - hs)/dx + 1)
        gxmax = int((xs[0] + hs)/dx)
        gymin = int((xs[1] - hs)/dx + 1)
        gymax = int((xs[1] + hs)/dx)
        
        for gx in xrange(gxmin, gxmax+1):
            for gy in xrange(gymin,gymax+1):
                ix = gx%gridres
                iy = gy%gridres
                delta_x = np.abs(xs[0] - ix*dx)
                delta_x = min(delta_x, L-delta_x)
                delta_y = np.abs(xs[1] - iy*dx)
                delta_y = min(delta_y, L-delta_y)
                kernel = Kernel2D((delta_x**2 + delta_y**2)**0.5 / hs)
                grid[ix,iy] +=  kernel * mh2
                 
    return grid

In [ ]:
N = 30**3
#x = np.mgrid[0:0.99:30j,0:0.99:30j, 0:0.99:30j] + 0.5
#x = np.c_[x[0].flatten(),x[1].flatten(),x[2].flatten()]

#x = np.random.rand(N, 3)
#h = GetHsml(x,des_ngb=32, boxsize=1.0)
x = np.array([[0.0,0.0,0.0]])
h = np.array([0.25])
gridres = 100
h = np.clip(h,2./gridres,1e100)
L = 1.0
data = np.ones(N)/N

g1= IntegrateSurfaceDensityPeriodic(data, x, h, gridres, L)
g2= IntegrateSurfaceDensityPeriodic(data, x-0.5, h, gridres, L)
#print (g - g[:,::-1]).std()

plt.imshow(g2,cmap='viridis')
plt.colorbar()
plt.show()

In [31]:
print (g-g_orig).std()

0.0


In [123]:
x = np.linspace(0,1,100)
plt.plot(x, Kernel2D(x))
plt.plot(x,1.8189136353359467*np.exp(-8*x**2 / 2))
plt.show()